In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [10]:
# 파일 불러오기
df_0 = pd.read_csv('./train_0.csv')  
df_1 = pd.read_csv('./train_1.csv')

# 하나로 합치기
df = pd.concat([df_0, df_1], ignore_index=True)

# 순서 섞기
df = df.sample(frac=1).reset_index(drop=True)

# 입력값과 타깃 분리
X = df['title']
y = df['tag']

# 훈련/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 벡터화
vectorizer = CountVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 분류기 학습
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# 예측 및 평가
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8732    0.9185    0.8953       405
           1     0.9118    0.8633    0.8869       395

    accuracy                         0.8912       800
   macro avg     0.8925    0.8909    0.8911       800
weighted avg     0.8923    0.8912    0.8911       800



In [11]:
# 사용자 입력 받아 분류하는 함수
def predict_news_title(title: str):
    # 입력 문장을 벡터화
    title_vec = vectorizer.transform([title])
    
    # 예측
    prediction = model.predict(title_vec)[0]
    prob = model.predict_proba(title_vec)[0][prediction]
    
    # 결과 출력
    if prediction == 1:
        print(f"[노동 관련 뉴스] ({prob:.2f} 확률)")
    else:
        print(f"[노동 무관 뉴스] ({prob:.2f} 확률)")

In [16]:
# 예시 사용
user_input = input("뉴스 제목을 입력하세요: ")
predict_news_title(user_input)

[노동 관련 뉴스] (0.73 확률)


In [13]:
# 모델과 벡터라이저 저장
joblib.dump(model, 'logistic_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']